In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import torch
from torch import nn
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision.models import densenet121


train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


train_dataset = datasets.ImageFolder('drive/MyDrive/train', transform=train_transform)
test_dataset = datasets.ImageFolder('drive/MyDrive/test', transform=test_transform)
train_loader = DataLoader(train_dataset, batch_size=3, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=3, shuffle=False)


model = densenet121(pretrained=True)

num_ftrs = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_ftrs, 4)
)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-8, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    scheduler.step()
    train_accuracy = 100 * correct_train / total_train
    print(f"Epoch {epoch+1}, Train Loss: {running_loss / len(train_loader)}, Train Accuracy: {train_accuracy}%")


    model.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    test_accuracy = 100 * correct_test / total_test
    print(f'Epoch {epoch+1}, Test Accuracy: {test_accuracy}%')


Epoch 1, Train Loss: 1.0970601901775454, Train Accuracy: 49.75530179445351%
Epoch 1, Test Accuracy: 58.73015873015873%
Epoch 2, Train Loss: 0.8567864452193423, Train Accuracy: 65.25285481239804%
Epoch 2, Test Accuracy: 73.65079365079364%
Epoch 3, Train Loss: 0.6834803218340001, Train Accuracy: 75.53017944535074%
Epoch 3, Test Accuracy: 65.71428571428571%
Epoch 4, Train Loss: 0.6025495973120375, Train Accuracy: 76.3458401305057%
Epoch 4, Test Accuracy: 74.92063492063492%
Epoch 5, Train Loss: 0.5876357909019401, Train Accuracy: 78.95595432300163%
Epoch 5, Test Accuracy: 82.53968253968254%
Epoch 6, Train Loss: 0.4679651507789769, Train Accuracy: 84.33931484502448%
Epoch 6, Test Accuracy: 85.71428571428571%
Epoch 7, Train Loss: 0.4300973159601776, Train Accuracy: 84.99184339314846%
Epoch 7, Test Accuracy: 74.92063492063492%
Epoch 8, Train Loss: 0.39421346390211, Train Accuracy: 87.27569331158239%
Epoch 8, Test Accuracy: 87.61904761904762%
Epoch 9, Train Loss: 0.2770632506779781, Train Accu